## Sudoku

![Sudoku](http://upload.wikimedia.org/wikipedia/commons/f/ff/Sudoku-by-L2G-20050714.svg)

**Sudoku** is a number puzzle played on a 9x9 grid. The challenge is to place a digit between 1 and 9 inclusive in each empty cell, such that the completed grid obeys the following rules:

* Each row contains the numbers 1 to 9 once and only once.
* Each column contains the numbers 1 to 9 once and only once.
* Each 3x3 subgrid contains the numbers 1 to 9 once and only once.

The most natural formulation of this problem would probably be something like

$$x_{i,j} \in \{1, 2, \dots, 9\}$$

which is of course something we can do with integer programming:

$$1 \leq x_{i,j} \leq 9, ~ integer$$

The challenge now is the constraints. One observation is that the numbers 1 to 9 add up to 45, so we could try something like:

$$ \sum_{j=1}^9 x_{i,j} = 45 $$

And something similar for the columns and squares.

In [ ]:
init_vals = [
5 3 0 0 7 0 0 0 0
6 0 0 1 9 5 0 0 0
0 9 8 0 0 0 0 6 0
8 0 0 0 6 0 0 0 3
4 0 0 8 0 3 0 0 1
7 0 0 0 2 0 0 0 6
0 6 0 0 0 0 2 8 0
0 0 0 4 1 9 0 0 5
0 0 0 0 8 0 0 7 9
]

using JuMP, Gurobi
function sudoku1()
    sudoku = Model(solver=GurobiSolver(OutputFlag=0))
    @variable(sudoku, 1 <= x[1:9,1:9] <= 9, Int)
    @constraint(sudoku, rows[i=1:9], sum(x[i,:]) == 45)
    @constraint(sudoku, cols[j=1:9], sum(x[:,j]) == 45)
    @constraint(sudoku,subgrid[i=1:3:7,j=1:3:7], sum(x[i:i+2,j:j+2]) == 45)
    for i in 1:9, j in 1:9
        if init_vals[i,j] != 0
            @constraint(sudoku, x[i,j] == init_vals[i,j])
        end
    end
    
    solve(sudoku)
    
    Int.(getvalue(x))
end
grid1 = sudoku1()

Well that didn't work. Our model can't have been correct. This shows the importance of validating the output from an optimization model to make sure it makes sense and actually reflects the problem you are trying to solve.

## Exercise

Let's write a function to validate any sudoku solution and check if it is valid:

In [ ]:
function check_sudoku(grid)
    # TODO: Your code here
end    

Let's try it out:

In [ ]:
check_sudoku(grid1)

## New formulation

Let's fix our formulation so it is correct. We can change our **variables**: $x_{i,j,k} = 1$ iff the number $k$ will appear in cell $(i,j)$. We can now use our 0-1 integer programming knowledge to model the problem. Consider the "row" constraints: we want each number to appear once and only once. This is equivalent to saying that

$$\sum_{j=1}^9 x_{i,j,k} = 1 \quad \forall i, k$$

We can now model this as a $9\times 9\times 9$ dimensional problem - thats a lot of binary variables, surely Gurobi won't like that!

In [ ]:
function sudoku2()
    sudoku = Model(solver=GurobiSolver())

    @variable(sudoku, x[i=1:9, j=1:9, k=1:9], Bin)

    @constraints(sudoku, begin
        # TODO: Add constraints here...
    end)

    # TODO: Set initial solution here...
    
    solve(sudoku)
    
    getvalue(x)
end
grid2 = sudoku2()

We can check if this is a valid solution, but we have to change it into the grid format first!

In [ ]:
grid2a = zeros(Int, 9, 9)
for i = 1:9
    for j = 1:9
        grid2a[i, j] = findfirst(grid2[i, j, :])
    end
end
grid2a

In [ ]:
check_sudoku(grid2a)

It works!